### Librerías

In [1]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Conexión SQL
# ==============================================================================
import psycopg2 as ps

import seaborn as sns

#Biblioteca para estimar modelos estadísticos y realizar pruebas estadísticas.
# ==============================================================================
import statsmodels.formula.api as sfm 

from sklearn.preprocessing import PolynomialFeatures 

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

from sklearn import linear_model

### Conexión SQL

In [2]:
conexion = ps.connect(host="codbview.c21jlbp5d5qr.us-east-1.rds.amazonaws.com", database="codb", user="sgonzalez", password="AdarlaToda100%",port="5432")

In [3]:
# Creamos el cursor con el objeto conexion
cur = conexion.cursor()

# Ejecutamos una consulta
sql = """
select *
from sal_indicators_in sa 
"""

consulta = cur.execute(sql)

data = cur.fetchall()

In [5]:
df = pd.DataFrame(data,columns=['date_char','day_id','sales_channel','ipc','activations','activations_w_topups','activations_pacs','topups','total_pacs','radio','tv','ooh','btl_distri','btl_kioscos_y_retail','pop','carro_valla','digital_performance','rotacion','residual','avg_recarga_residual','comisiones','eventos_alto_bitacora','eventos_bajo_bitacora','eventos_medio_bitacora','ind_competividad','dias_lab'])

df['ipc']=df['ipc'].astype('float')
df['rotacion']=df['rotacion'].astype('float')
df['residual']=df['residual'].astype('float')
df['avg_recarga_residual']=df['avg_recarga_residual'].astype('float')
df['eventos_alto_bitacora']=df['eventos_alto_bitacora'].astype('float')
df['eventos_bajo_bitacora']=df['eventos_bajo_bitacora'].astype('float')
df['eventos_medio_bitacora']=df['eventos_medio_bitacora'].astype('float')

df = df.fillna(0)
#df = df.dropna()

df.head()

,date_char,day_id,sales_channel,ipc,activations,activations_w_topups,activations_pacs,topups,total_pacs,radio,...,digital_performance,rotacion,residual,avg_recarga_residual,comisiones,eventos_alto_bitacora,eventos_bajo_bitacora,eventos_medio_bitacora,ind_competividad,dias_lab
0,jun-21,2021-01-06,Others,4.0,1110,91,86,2083000,2522,142308892,...,393863417,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0.83,24
1,ene-21,2021-01-01,Distributors - Traditional,1.6,40265,38311,17299,115018055,349104,41687525,...,365633245,0.00,0.21,4469.0,0.0,0.0,0.0,1.0,1.03,24
2,ene-21,2021-01-01,Retail,1.6,16482,16307,15328,265657864,187343,41687525,...,365633245,0.06,0.00,0.0,40858121.0,0.0,0.0,1.0,1.03,24
3,ene-21,2021-01-01,Kioskos,1.6,6128,6056,5546,73900256,86862,41687525,...,365633245,0.13,0.00,0.0,5355069.0,0.0,0.0,1.0,1.03,24
4,ene-21,2021-01-01,Digital,1.6,9566,5461,5364,128270496,40254,41687525,...,365633245,0.00,0.00,0.0,0.0,0.0,0.0,1.0,1.03,24


In [6]:
df.to_csv(r'C:\Users\juan.gonzalez\Downloads\Prueba Grifo.csv')

### Distribución de los Datos

In [ ]:
plt.figure(figsize=(30,20)) 
sns.set_theme(style="ticks")

sns.set(font_scale = 2)

#sns.pairplot(df, x_vars=['ipc','radio','tv','ooh','btl_distri','btl_kioscos_y_retail','pop','carro_valla','digital_performance','rotacion','residual','avg_recarga_residual','comisiones','eventos_alto_bitacora','eventos_bajo_bitacora','eventos_medio_bitacora'], y_vars=['activations']) 
sns.pairplot(df, x_vars=['ipc','radio'], y_vars=['activations'],height=5)


                         
plt.show()

### Correlaciones

In [ ]:
plt.figure(figsize=(3,6)) 

sns.set(font_scale = 1)

corrMatrix_1 = df.corr(method ='spearman')[['activations']]


sns.heatmap(corrMatrix_1, annot=True,cmap="YlGnBu")

plt.show()

In [ ]:
del df['date_char']
del df['day_id']
del df['sales_channel']
del df['activations_w_topups']
del df['activations_pacs']
del df['topups']
del df['total_pacs']
del df['ind_productividad']
del df['dias_lab']

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))

scaled = scaler.fit_transform(df)

df_normal = pd.DataFrame(scaled,columns=['ipc','activations','radio','tv','ooh','btl_distri','btl_kioscos_y_retail','pop','carro_valla','digital_performance','rotacion','residual','avg_recarga_residual','comisiones','eventos_alto_bitacora','eventos_bajo_bitacora','eventos_medio_bitacora'])

df_normal['ipc']=df_normal['ipc'].astype('float')
df_normal['rotacion']=df_normal['rotacion'].astype('float')
df_normal['residual']=df_normal['residual'].astype('float')
df_normal['avg_recarga_residual']=df_normal['avg_recarga_residual'].astype('float')
df_normal['eventos_alto_bitacora']=df_normal['eventos_alto_bitacora'].astype('float')
df_normal['eventos_bajo_bitacora']=df_normal['eventos_bajo_bitacora'].astype('float')
df_normal['eventos_medio_bitacora']=df_normal['eventos_medio_bitacora'].astype('float')

df_normal = df_normal.fillna(0)
#df = df.dropna()

df_normal.head()

### Omitir Variables 
De acuerdo a su correlación

In [ ]:
del df['date_char']
del df['day_id']
del df['sales_channel']
del df['activations_w_topups']
del df['activations_pacs']
del df['topups']
del df['total_pacs']


#del df['eventos_medio_bitacora']
#del df['ipc']
#del df['eventos_alto_bitacora']
#del df['ooh']
#del df['pop']
#del df['tv']
#del df['rotacion']
#del df['comisiones']
#del df['digital_performance']
#del df['eventos_bajo_bitacora']

### Regresión Lineal

In [ ]:
# División de los datos en train y test
# ==============================================================================
X = df.drop(columns=['activations'],axis=1)
y = df['activations']

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

#Prediction using test set 
y_pred = lin_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred)
mse=metrics.mean_squared_error(y_test, y_pred)

# Printing the metrics
print('R2 square:',metrics.r2_score(y_test, y_pred))
print('MAE: ', mae)
print('MSE: ', mse)
print('')

print('explained_variance:',metrics.explained_variance_score(y_test, y_pred))
print('')

print('')
model=sm.OLS(y,X).fit() 

model.summary()

In [ ]:
print(X.columns)
print("")
print('w = ' + str(lin_reg.coef_) + ', b = ' + str(lin_reg.intercept_))

In [ ]:
X.columns

In [ ]:
np.array(lin_reg.coef_).T


In [ ]:
#np.std(X)

X.std()

In [ ]:
X2 = df.drop(columns=['activations'],axis=1)
y2 = df['activations']


prueba = pd.DataFrame(X.columns,columns=['columnas'])

prueba['coef'] = np.array(lin_reg.coef_).T.astype('float')
prueba['m'] = np.array(X2.mean().values).T.astype('float')
prueba['std'] = np.array(X2.std().values).T.astype('float')



prueba.head(50)

In [ ]:
prueba.to_csv(r'C:\Users\juan.gonzalez\Downloads\prueba cor.csv')

\begin{align}
\dot{activations} = \ -ipc(1.08707071e+03) - radio(1.69505491e-05) - tv(8.75945245e-07) + ooh(5.84508790e-06) + btlDistri(9.75854623e-05) - btlKioscosYretail(6.32994351e-05) + pop(5.41916990e-06) - carroValla(3.38964506e-04) - digitalPerformance(1.70410130e-06) + rotacion(2.02202299e+04) + residual(3.82600547e+04) + avgRecargaResidual(3.96941585e+00) + comisiones(1.36926477e-04) - eventosAltoBitacora(1.57219220e+05) - eventosBajoBitacora(1.59499580e+05) - eventosMedioBitacora(1.57618636e+05) + 166688.52349714\\
\end{align}

In [ ]:
model.params

### Modelo Regresión Activaciones con Recarga y con PAC

In [ ]:
# Creamos el cursor con el objeto conexion
cur2 = conexion.cursor()

# Ejecutamos una consulta
sql2 = """
select activations,activations_w_topups, activations_pacs
eventos_medio_bitacora
from sal_indicators_in sa 
"""

consulta2 = cur2.execute(sql2)

data2 = cur2.fetchall()

In [ ]:
df2 = pd.DataFrame(data2,columns=['activations','activations_w_topups','activations_pacs'])



df2 = df2.fillna(0)
#df = df.dropna()

df2.head()

##### Recargas

In [ ]:
# División de los datos en train y test
# ==============================================================================

X = df2[['activations']]
y = df2['activations_w_topups']


X_train, X_test, y_train, y_test = train_test_split(
                                        X.values.reshape(-1,1),
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

#Prediction using test set 
y_pred = lin_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred)
mse=metrics.mean_squared_error(y_test, y_pred)

# Printing the metrics
print('R2 square:',metrics.r2_score(y_test, y_pred))
print('MAE: ', mae)
print('MSE: ', mse)
print('')

print('explained_variance:',metrics.explained_variance_score(y_test, y_pred))
print('')

print('')
model=sm.OLS(y,X).fit() 

model.summary()

In [ ]:
print(X.columns)
print("")
print('w = ' + str(lin_reg.coef_) + ', b = ' + str(lin_reg.intercept_))

\begin{align}
\dot{activations W topups} = \ activations(1.0011686) - 359.57254012\\
\end{align}

##### PACS

In [ ]:
# División de los datos en train y test
# ==============================================================================
X = df2[['activations_w_topups']]
y = df2['activations_pacs']

X_train, X_test, y_train, y_test = train_test_split(
                                        X.values.reshape(-1,1),
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

#Prediction using test set 
y_pred = lin_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred)
mse=metrics.mean_squared_error(y_test, y_pred)

# Printing the metrics
print('R2 square:',metrics.r2_score(y_test, y_pred))
print('MAE: ', mae)
print('MSE: ', mse)
print('')

print('explained_variance:',metrics.explained_variance_score(y_test, y_pred))
print('')

print('')
model=sm.OLS(y,X).fit() 

model.summary()

In [ ]:
print(X.columns)
print("")
print('w = ' + str(lin_reg.coef_) + ', b = ' + str(lin_reg.intercept_))

\begin{align}
\dot{activations PACs} = \ activationsWtopups(0.49195756) + 1624.23223709\\
\end{align}